<a href="https://colab.research.google.com/github/FUJITOSHION/transformer/blob/master/fall_zemi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 候補
- refomerを行う => transfomerと比較
- 同等の精度で計算量の違いを表す。

## transfomerの問題点

attentionの計算量が多い
特に内積QK
refomerではlocality-sensitive hashing(LSH)にを使う。

## 注意
実際のtransformerはMluti-Head-attentionだが今回はsimple transfomerを使う。


## Transformerの構造
1. 文章の単語ID列
2. Embedderモジュール(word_id to word_vec)
3. positionalEncoder
4. TranformerBlock * 2
    1. LayerNormalization
    2. Attention(self-attention)
    3. Dropout
    4. Layer Normalization
    5. FeedForwad
    6. DropOut
5. clasificationHead

In [ ]:
!nvidia-smi

Tue Jan 26 09:40:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from model import TransformerClassification
from train_model import train_model
import ipython_ready as ir
import torch.nn as nn
import torch.optim as optim


dl_class = ir.GetDataLoader()
dl_dict, TEXT = dl_class()
net = TransformerClassification(TEXT.vocab.vectors)


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        # Liner層の初期化
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)


# 訓練モードに設定
net.train()

# TransformerBlockモジュールを初期化実行
net.net3_1.apply(weights_init)
net.net3_2.apply(weights_init)

num_epochs = 10
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

# 最適化手法の設定
learning_rate = 2e-5
optimizer = optim.Adam(net.parameters(), lr=learning_rate)
net_trained = train_model(net, dl_dict,
                          criterion, optimizer, num_epochs=num_epochs)

---------------
finish make folder
---------------
---------------
get imdb
---------------
---------------
open tar file
---------------
---------------
to tsv
---------------
---------------
finish dataloader
---------------
---------------
finish make folder
---------------
---------------
get imdb
---------------
---------------
open tar file
---------------
---------------
to tsv
---------------
---------------
finish dataloader
---------------
使用デバイス： cuda:0
-----start-------
Epoch 1/10 | train |  Loss: 0.5641 Acc: 0.7004
Epoch 1/10 |  val  |  Loss: 0.4205 Acc: 0.8090
Epoch 2/10 | train |  Loss: 0.4207 Acc: 0.8129
Epoch 2/10 |  val  |  Loss: 0.3932 Acc: 0.8196
Epoch 3/10 | train |  Loss: 0.3898 Acc: 0.8293
Epoch 3/10 |  val  |  Loss: 0.3676 Acc: 0.8398
Epoch 4/10 | train |  Loss: 0.3702 Acc: 0.8365
Epoch 4/10 |  val  |  Loss: 0.3783 Acc: 0.8354
Epoch 5/10 | train |  Loss: 0.3562 Acc: 0.8460
Epoch 5/10 |  val  |  Loss: 0.3597 Acc: 0.8456
Epoch 6/10 | train |  Loss: 0.3454 Acc: 0.8

In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)

epoch_corrects = 0  # epochの正解数

for batch in (dl_dict['test']):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    
    # GPUが使えるならGPUにデータを送る
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # mask作成
        input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
        input_mask = (inputs != input_pad)

        # Transformerに入力
        outputs, _, _ = net_trained(inputs, input_mask)
        _, preds = torch.max(outputs, 1)  # ラベルを予測

        # 結果の計算
        # 正解数の合計を更新
        epoch_corrects += torch.sum(preds == labels.data)

# 正解率
epoch_acc = epoch_corrects.double() / len(dl_dict['test'].dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(dl_dict['test'].dataset),epoch_acc))

テストデータ25000個での正解率：0.8449


In [ ]:
def highlight(word, attn):
    "Attentionの値が大きいと文字の背景が濃い赤になるhtmlを出力させる関数"

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normlized_weights_1, normlized_weights_2, TEXT):
    "HTMLデータを作成する"

    # indexの結果を抽出
    sentence = batch.Text[0][index]  # 文章
    label = batch.Label[index]  # ラベル
    pred = preds[index]  # 予測

    # indexのAttentionを抽出と規格化
    attens1 = normlized_weights_1[index, 0, :]  # 0番目の<cls>のAttention
    attens1 /= attens1.max()

    attens2 = normlized_weights_2[index, 0, :]  # 0番目の<cls>のAttention
    attens2 /= attens2.max()

    # ラベルと予測結果を文字に置き換え
    if label == 0:
        label_str = "Negative"
    else:
        label_str = "Positive"

    if pred == 0:
        pred_str = "Negative"
    else:
        pred_str = "Positive"

    # 表示用のHTMLを作成する
    html = '正解ラベル：{}<br>推論ラベル：{}<br><br>'.format(label_str, pred_str)

    # 1段目のAttention
    html += '[TransformerBlockの1段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens1):
        html += highlight(TEXT.vocab.itos[word], attn)
    html += "<br><br>"

    # 2段目のAttention
    html += '[TransformerBlockの2段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens2):
        html += highlight(TEXT.vocab.itos[word], attn)

    html += "<br><br>"

    return html

In [ ]:

from IPython.display import HTML

# Transformerで処理

# ミニバッチの用意
batch = next(iter(dl_dict['test']))

# GPUが使えるならGPUにデータを送る
inputs = batch.Text[0].to(device)  # 文章
labels = batch.Label.to(device)  # ラベル

# mask作成
input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
input_mask = (inputs != input_pad)

# Transformerに入力
outputs, normlized_weights_1, normlized_weights_2 = net_trained(
    inputs, input_mask)
_, preds = torch.max(outputs, 1)  # ラベルを予測


index = 3  # 出力させたいデータ
html_output = mk_html(index, batch, preds, normlized_weights_1,
                      normlized_weights_2, TEXT)  # HTML作成
HTML(html_output)  # HTML形式で出力

In [ ]:
index = 3  # 出力させたいデータ
attens1 = normlized_weights_1[index, 0, :]  # 0番目の<cls>のAttention
attens1 /= attens1.max()   
print(attens1)
print(attens1.size())

tensor([6.1128e-03, 4.7590e-03, 3.1670e-02, 4.9423e-03, 4.2871e-04, 5.7774e-03,
        8.8674e-03, 4.5309e-03, 3.5266e-02, 2.4184e-02, 2.5419e-02, 1.0000e+00,
        2.5713e-03, 4.9521e-02, 1.2750e-02, 1.7613e-01, 3.0343e-03, 2.5281e-04,
        2.3584e-03, 1.6890e-03, 2.7067e-04, 4.0156e-03, 3.8411e-03, 3.7079e-02,
        1.0446e-01, 3.1033e-04, 4.3010e-01, 2.5444e-03, 2.4953e-04, 1.5983e-02,
        1.4341e-03, 6.2758e-03, 2.9237e-03, 4.1862e-02, 4.2955e-02, 3.6582e-03,
        1.2170e-01, 3.0919e-03, 7.8523e-04, 1.5065e-01, 7.3490e-03, 1.7011e-02,
        1.1738e-02, 7.8737e-02, 1.0519e-02, 4.4515e-03, 1.8613e-01, 3.6508e-03,
        2.4070e-02, 3.7291e-02, 7.7643e-03, 6.7031e-01, 1.0160e-01, 4.1716e-02,
        4.8489e-03, 1.0528e-02, 2.9729e-01, 1.0994e-01, 5.0481e-03, 2.5245e-02,
        4.7463e-04, 6.6482e-03, 3.4962e-02, 1.4773e-02, 2.3497e-03, 2.6193e-01,
        9.0134e-03, 5.2069e-02, 3.1493e-03, 1.7598e-03, 6.7782e-04, 3.1541e-02,
        4.3174e-02, 3.5605e-03, 8.1097e-

In [ ]:
index = 24  # 出力させたいデータ
html_output = mk_html(index, batch, preds, normlized_weights_1,
                      normlized_weights_2, TEXT)  # HTML作成
HTML(html_output)  # HTML形式で出力